# Raw Task Injection

This notebook demonstrates how to wrap pre-configured `nidaqmx.task.Task` objects with nidaqwrapper classes.

**When to use raw task injection:**
- You have existing nidaqmx code that creates tasks
- You need advanced configuration not exposed by the wrapper API
- You want to interoperate with legacy codebases

All four task classes (AITask, AOTask, DITask, DOTask) provide a `from_task()` classmethod that wraps external tasks while respecting ownership semantics.

## Setup: Hardware Detection and Mock Fallback

In [ ]:
import numpy as np
import warnings

# Try to import nidaqmx — if unavailable, create a mock
HW_AVAILABLE = False

try:
    import nidaqmx
    from nidaqmx import constants
    
    # Check if hardware is actually connected
    system = nidaqmx.system.System.local()
    if len(system.devices) > 0:
        HW_AVAILABLE = True
        print(f"Real hardware detected: {[d.name for d in system.devices]}")
    else:
        raise ImportError("nidaqmx available but no devices connected")
        
except ImportError:
    print("No hardware available — using mock nidaqmx")
    
    # Create a minimal mock for demonstration purposes
    class MockChannel:
        def __init__(self, name):
            self.name = name
    
    class MockChannelCollection:
        def __init__(self, prefix):
            self.prefix = prefix
            self._channels = []
        
        def add_ai_voltage_chan(self, *args, **kwargs):
            ch = MockChannel(f"{kwargs.get('name_to_assign_to_channel', 'ai0')}")
            self._channels.append(ch)
        
        def add_ao_voltage_chan(self, *args, **kwargs):
            ch = MockChannel(f"{kwargs.get('name_to_assign_to_channel', 'ao0')}")
            self._channels.append(ch)
        
        def add_di_chan(self, *args, **kwargs):
            ch = MockChannel(f"{kwargs.get('name_to_assign_to_lines', 'di0')}")
            self._channels.append(ch)
        
        def add_do_chan(self, *args, **kwargs):
            ch = MockChannel(f"{kwargs.get('name_to_assign_to_lines', 'do0')}")
            self._channels.append(ch)
        
        def __len__(self):
            return len(self._channels)
        
        def __iter__(self):
            return iter(self._channels)
    
    class MockTiming:
        def __init__(self):
            self.samp_clk_rate = 1000.0
            self.samp_quant_samp_mode = None
        
        def cfg_samp_clk_timing(self, rate, sample_mode, **kwargs):
            self.samp_clk_rate = rate
            self.samp_quant_samp_mode = sample_mode
    
    class MockTask:
        def __init__(self, name="mock_task"):
            self.name = name
            self.ai_channels = MockChannelCollection("ai")
            self.ao_channels = MockChannelCollection("ao")
            self.di_channels = MockChannelCollection("di")
            self.do_channels = MockChannelCollection("do")
            self.timing = MockTiming()
            self.channel_names = []
        
        def start(self):
            pass
        
        def stop(self):
            pass
        
        def close(self):
            pass
        
        def is_task_done(self):
            return True
        
        def read(self, *args, **kwargs):
            return [0.0] * len(self.ai_channels)
    
    # Create a mock nidaqmx module
    class MockNidaqmx:
        class task:
            Task = MockTask
    
    nidaqmx = MockNidaqmx()

print(f"\nHW_AVAILABLE = {HW_AVAILABLE}")

## 1. AITask.from_task() — Wrapping Analog Input

Create a raw nidaqmx.Task, configure it manually, then wrap it with `AITask.from_task()`.

In [ ]:
from nidaqwrapper import AITask

# Create a raw nidaqmx task
raw_task = nidaqmx.task.Task("external_ai")

# Configure channels and timing directly on the raw task
if HW_AVAILABLE:
    raw_task.ai_channels.add_ai_voltage_chan(
        "Dev1/ai0",
        name_to_assign_to_channel="voltage_in"
    )
    raw_task.timing.cfg_samp_clk_timing(rate=25600)
else:
    # Mock mode: simulate channel addition
    raw_task.ai_channels.add_ai_voltage_chan(
        "Dev1/ai0",
        name_to_assign_to_channel="voltage_in"
    )
    raw_task.channel_names = ["voltage_in"]
    raw_task.timing.cfg_samp_clk_timing(rate=25600, sample_mode=None)

# Wrap with AITask.from_task()
wrapped = AITask.from_task(raw_task)

print(f"Wrapped task name: {wrapped.task_name}")
print(f"Sample rate: {wrapped.sample_rate} Hz")
print(f"Channels: {wrapped.channel_list}")
print(f"Owns task: {wrapped._owns_task}  (False = caller must close)")

# You can read data through the wrapper
if HW_AVAILABLE:
    raw_task.start()
    data = wrapped.acquire_base()  # Returns (n_channels, n_samples)
    print(f"\nData shape: {data.shape}")
    raw_task.stop()
else:
    print("\nSkipping acquire_base() in mock mode")

# add_channel() and start() are BLOCKED for external tasks
try:
    wrapped.add_channel("illegal", device_ind=0, channel_ind=1, units="V")
except RuntimeError as e:
    print(f"\nExpected error on add_channel(): {e}")

try:
    wrapped.start()
except RuntimeError as e:
    print(f"Expected error on start(): {e}")

# clear_task() does NOT close the task — warns instead
print("\nCalling wrapped.clear_task()...")
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    wrapped.clear_task()
    if w:
        print(f"Warning: {w[0].message}")

# Caller must close the raw task
raw_task.close()
print("Closed raw_task (caller responsibility)")

## 2. AOTask.from_task() — Wrapping Analog Output

Same pattern for analog output tasks.

In [ ]:
from nidaqwrapper import AOTask

# Create a raw nidaqmx task for AO
raw_ao_task = nidaqmx.task.Task("external_ao")

if HW_AVAILABLE:
    raw_ao_task.ao_channels.add_ao_voltage_chan(
        "Dev1/ao0",
        name_to_assign_to_channel="signal_out"
    )
    raw_ao_task.timing.cfg_samp_clk_timing(rate=10000)
else:
    raw_ao_task.ao_channels.add_ao_voltage_chan(
        "Dev1/ao0",
        name_to_assign_to_channel="signal_out"
    )
    raw_ao_task.channel_names = ["signal_out"]
    raw_ao_task.timing.cfg_samp_clk_timing(rate=10000, sample_mode=None)

# Wrap it
wrapped_ao = AOTask.from_task(raw_ao_task)

print(f"Wrapped AO task: {wrapped_ao.task_name}")
print(f"Sample rate: {wrapped_ao.sample_rate} Hz")
print(f"Channels: {wrapped_ao.channel_list}")
print(f"Owns task: {wrapped_ao._owns_task}")

# generate() works — writes data to the raw task
if HW_AVAILABLE:
    signal = np.sin(2 * np.pi * 1000 * np.linspace(0, 1, 10000))
    wrapped_ao.generate(signal)
    print("\nGenerated signal (10000 samples)")
else:
    print("\nSkipping generate() in mock mode")

# Cleanup: clear_task() warns, caller closes
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    wrapped_ao.clear_task()
    if w:
        print(f"\nWarning: {w[0].message}")

raw_ao_task.close()
print("Closed raw_ao_task")

## 3. DITask.from_task() — Wrapping Digital Input

Digital tasks follow the same ownership semantics.

In [ ]:
from nidaqwrapper import DITask

# Create raw DI task
raw_di_task = nidaqmx.task.Task("external_di")

if HW_AVAILABLE:
    raw_di_task.di_channels.add_di_chan("Dev1/port0/line0:3")
    raw_di_task.timing.cfg_samp_clk_timing(rate=1000)
else:
    raw_di_task.di_channels.add_di_chan("Dev1/port0/line0:3")
    raw_di_task.channel_names = ["port0/line0", "port0/line1", "port0/line2", "port0/line3"]
    raw_di_task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=None)

# Wrap it
wrapped_di = DITask.from_task(raw_di_task)

print(f"Wrapped DI task: {wrapped_di.task_name}")
print(f"Mode: {wrapped_di.mode}")
print(f"Channels: {wrapped_di.channel_list}")
print(f"Owns task: {wrapped_di._owns_task}")

# add_channel() and start() are blocked
try:
    wrapped_di.add_channel("illegal", lines="Dev1/port1/line0")
except RuntimeError as e:
    print(f"\nExpected error: {e}")

# Cleanup
wrapped_di.clear_task()  # Warns
raw_di_task.close()
print("\nClosed raw_di_task")

## 4. DOTask.from_task() — Wrapping Digital Output

Digital output follows the same pattern.

In [ ]:
from nidaqwrapper import DOTask

# Create raw DO task
raw_do_task = nidaqmx.task.Task("external_do")

if HW_AVAILABLE:
    raw_do_task.do_channels.add_do_chan("Dev1/port1/line0:7")
else:
    raw_do_task.do_channels.add_do_chan("Dev1/port1/line0:7")
    raw_do_task.channel_names = [f"port1/line{i}" for i in range(8)]

# Wrap it
wrapped_do = DOTask.from_task(raw_do_task)

print(f"Wrapped DO task: {wrapped_do.task_name}")
print(f"Mode: {wrapped_do.mode}")
print(f"Channels: {wrapped_do.channel_list}")
print(f"Owns task: {wrapped_do._owns_task}")

# write() works — delegates to raw task
if HW_AVAILABLE:
    wrapped_do.write([True, False, True, False, True, False, True, False])
    print("\nWrote pattern to digital lines")
else:
    print("\nSkipping write() in mock mode")

# Cleanup
wrapped_do.clear_task()
raw_do_task.close()
print("\nClosed raw_do_task")

## 5. Passing Raw Tasks to DAQHandler

DAQHandler.configure() accepts raw nidaqmx.task.Task objects and auto-wraps them via from_task().

In [ ]:
from nidaqwrapper import DAQHandler

# Create raw tasks
raw_ai = nidaqmx.task.Task("handler_ai")
raw_ao = nidaqmx.task.Task("handler_ao")
raw_di = nidaqmx.task.Task("handler_di")

if HW_AVAILABLE:
    raw_ai.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    raw_ai.timing.cfg_samp_clk_timing(rate=1000)
    raw_ao.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    raw_ao.timing.cfg_samp_clk_timing(rate=1000)
    raw_di.di_channels.add_di_chan("Dev1/port0/line0:3")
else:
    raw_ai.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    raw_ai.channel_names = ["ai0"]
    raw_ai.timing.cfg_samp_clk_timing(rate=1000, sample_mode=None)
    raw_ao.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    raw_ao.channel_names = ["ao0"]
    raw_ao.timing.cfg_samp_clk_timing(rate=1000, sample_mode=None)
    raw_di.di_channels.add_di_chan("Dev1/port0/line0:3")
    raw_di.channel_names = ["di0"]

# Pass raw tasks to DAQHandler — they are auto-wrapped
handler = DAQHandler()
handler.configure(
    task_in=raw_ai,
    task_out=raw_ao,
    task_digital_in=raw_di
)

print("DAQHandler configured with raw tasks")
print(f"Input task: {handler._task_in_is_obj} (wrapped to AITask)")
print(f"Output task: {handler._task_out_is_obj} (wrapped to AOTask)")
print(f"Digital input task: {handler._task_digital_in_is_obj} (wrapped to DITask)")

# connect() starts the wrapped tasks
if HW_AVAILABLE:
    if handler.connect():
        print("\nConnected to hardware")
        # Use the handler normally
        handler.disconnect()
        print("Disconnected")
else:
    print("\nSkipping connect() in mock mode")

# Cleanup: DAQHandler does NOT close external tasks
# Caller must close them
raw_ai.close()
raw_ao.close()
raw_di.close()
print("\nClosed all raw tasks (caller responsibility)")

## Ownership Semantics Summary

When a task is created via `from_task()`:

| Method | Behavior |
|--------|----------|
| `_owns_task` | `False` — wrapper does NOT own the task |
| `add_channel()` | Raises `RuntimeError` — channels must be configured before wrapping |
| `start()` | Raises `RuntimeError` — timing must be configured before wrapping |
| `clear_task()` | Does NOT close the task — emits a warning |
| `__exit__()` | Does NOT close the task — respects ownership |
| Caller responsibility | Must call `task.close()` when done |

This ensures that external tasks are never accidentally closed by the wrapper.

## Use Cases

**1. Interop with existing nidaqmx code:**
```python
# Legacy code creates the task
task = legacy_setup_function()
# Wrap it to use nidaqwrapper APIs
wrapped = AITask.from_task(task)
data = wrapped.acquire_base()
# Legacy code cleans up
task.close()
```

**2. Advanced configuration not exposed by wrapper:**
```python
task = nidaqmx.task.Task()
task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
# Set advanced properties not in AITask API
task.ai_channels[0].ai_coupling = constants.Coupling.DC
task.timing.cfg_samp_clk_timing(rate=1000)
# Wrap and use wrapper methods
wrapped = AITask.from_task(task)
data = wrapped.acquire_base()
task.close()
```

**3. Mixing raw and wrapped tasks in DAQHandler:**
```python
raw_input = nidaqmx.task.Task()
# ... configure raw_input ...
programmatic_output = AOTask("output", sample_rate=1000)
programmatic_output.add_channel("ao0", device_ind=0, channel_ind=0)
handler = DAQHandler(
    task_in=raw_input,        # Auto-wrapped
    task_out=programmatic_output  # Used directly
)
```